In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from arc import *
from time import time
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from typing import List, Dict

matplotlib.rc('font', family='sans-serif') 

from basics import *
from utility import *
from floquet_hamiltonian import *

pi=np.pi
tau=2*pi

# Plot old measured polarizabilities

In [2]:
exp_pol = pd.read_csv("Pin1_4780MHz_Polarizabilities.csv")

results_dir = r"Shirley-Floquet_results\PolarizabilityPlots\RawData"
plots_dir =  r"Shirley-Floquet_results\PolarizabilityPlots"

In [3]:
exp_pol

,RF power (dBm),RF Frequency (MHz),Effective Polarizability 0 (MHz/V^2),Error 0 (MHz/V^2),Effective Polarizability 1 (MHz/V^2),Error 1 (MHz/V^2),Effective Polarizability 2 (MHz/V^2),Error 2 (MHz/V^2),Effective Polarizability 3 (MHz/V^2),Error 3 (MHz/V^2)
0,-100.0,4780,-3.51,0.04,NaN,NaN,NaN,NaN,NaN,NaN
1,-17.0,4780,-2.40,0.05,-2.78,0.060,NaN,NaN,-2.33,0.03
2,-15.0,4780,-2.46,0.09,-2.76,0.140,NaN,NaN,-2.26,0.19
3,-14.0,4780,-2.36,0.04,-2.44,0.160,NaN,NaN,-2.17,0.13
4,-11.0,4780,-2.02,0.08,-2.07,0.150,-2.22,0.16,-2.32,0.15
5,-9.0,4780,-1.97,0.10,-1.85,0.470,-1.95,0.03,-2.26,0.47
6,-6.5,4780,-1.53,0.17,-1.00,0.272,-1.62,0.52,-1.96,0.35


In [4]:
x_dat = exp_pol["RF power (dBm)"]
y_str = "Effective Polarizability {} (MHz/V^2)"
er_str = "Error {} (MHz/V^2)"
fig, ax = plt.subplots(1,1)
for band in range(4):
    y_dat = exp_pol[y_str.format(band)]
    y_err = exp_pol[er_str.format(band)]
    ax.errorbar(x_dat,y_dat,yerr=y_err,fmt=".",label = f"Band {band}")
ax.legend()
ax.set_xlabel("RF power (dBm)")
ax.set_ylabel("Measured Effective Polarizability ($MHz/V^2$)")
fig.show()

<IPython.core.display.Javascript object>

In [5]:
x_dat_dB = exp_pol["RF power (dBm)"]
x_dat_scale = np.sqrt(10**(x_dat_dB/10))
y_str = "Effective Polarizability {} (MHz/V^2)"
er_str = "Error {} (MHz/V^2)"
fig, ax = plt.subplots(1,1)
for band in range(4):
    y_dat = exp_pol[y_str.format(band)]
    y_err = exp_pol[er_str.format(band)]
    ax.errorbar(x_dat_scale+band*2e-4,y_dat,yerr=y_err,fmt=".",label = f"Band {band}")
ax.legend()
ax.set_xlabel("RF power (Proportional to $(V/m)^2$)")
ax.set_ylabel("$-2\\alpha'$ on Pin 1 ($MHz/V^2$)")
fig.show()

<IPython.core.display.Javascript object>

# Load AC stark shift datasets

In [6]:
# Load datasets

# Taken with 684nm +80MHz from 4-6' transition
dAC_1 = pd.read_csv("ExperimentData/AC_stark_m30-m23dBm-m50MHz.csv")
dAC_2 = pd.read_csv("ExperimentData/AC_stark_m23dBm-m10dBm-m50MHz.csv")

# Taken with 684nm +60MHz from 4-6' transition
'''
# these datasets excluded to make use of files with GOOD data included
dAC_4 = pd.read_csv("ExperimentData/AC_stark_m10dBm-m6dBm-m50MHz-2022-01-24.csv")
dAC_5 = pd.read_csv("ExperimentData/AC_stark_m30dBm-m25dBm-m50MHz-2022-01-25.csv")
dAC_6 = pd.read_csv("ExperimentData/AC_stark_m25dBm-m18dBm-m50MHz-2022-01-26.csv")
'''
# GOOD logger inclusive versions of above datasets
dAC_4 = pd.read_csv("ExperimentData/AC_stark_m16dBm-m8dBm-m50MHz-2022-01-24-GOOD.csv")
dAC_5 = pd.read_csv("ExperimentData/AC_stark_m30dBm-m25dBm-m50MHz-2022-01-25-GOOD.csv")
dAC_6 = pd.read_csv("ExperimentData/AC_stark_m25dBm-m18dBm-m50MHz-2022-01-26-GOOD.csv")
dAC_7 = pd.read_csv("ExperimentData/AC_stark_m12dBm-m6dBm-m50MHz-2022-01-27-GOOD.csv")

dAC_m20 = dAC_5.append(dAC_6)
dAC_m20 = dAC_m20.append(dAC_4[:-2])
dAC_m20 = dAC_m20.append(dAC_7)

dAC_m20[:-5]

drive_cal = pd.read_csv("ExperimentData/4780MHZ_pickoffPower.csv")

<ipython-input-6-285d23b650b9>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dAC_m20 = dAC_5.append(dAC_6)
<ipython-input-6-285d23b650b9>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dAC_m20 = dAC_m20.append(dAC_4[:-2])
<ipython-input-6-285d23b650b9>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dAC_m20 = dAC_m20.append(dAC_7)


In [7]:
# Calibrate drive power
x_set = np.array(drive_cal["Set Power (dBm)"])
y_pk = np.array(drive_cal["Pick-off power (dBm)"])

cal_fun = interp1d(x_set, y_pk, kind="linear")

In [8]:
# Failed GOOD logger version of calibration
fn = lambda x, a, s: a*10**(float(s)*x)
fig,ax = plt.subplots(1,1)
ax.plot(dAC_m20["RydDressPower (dBm)"], np.abs(dAC_m20["Measured Dressing Power (arb)"]),".")
xln = np.linspace(min(dAC_m20["RydDressPower (dBm)"]), max(dAC_m20["RydDressPower (dBm)"]), 1000)
ax.plot(dAC_m20["RydDressPower (dBm)"],fn(dAC_m20["RydDressPower (dBm)"],100,0.07))
ax.set_xlabel("Set Power (dBm)")
ax.set_ylabel("Measured Power (arb)")
ax.set_yscale("log")
fig.show()

<IPython.core.display.Javascript object>

In [9]:
# Build dataframe that concatenates all data taken at +80MHz intermediate detuning

x_ignores = 2  # ignoring the first n values from the dataset in dAC_2
x_data = np.array(dAC_1["RydDressPower (dBm)"])
x_data = np.append(x_data,dAC_2["RydDressPower (dBm)"][x_ignores:])
print(x_data)

n_bands = 4
dACs = np.zeros((n_bands,len(x_data)),dtype=float)
dACs_er = np.zeros(dACs.shape,dtype=float)

# add data from dataset 1
for band in range(n_bands):
    dACs[band,:len(dAC_1)] = dAC_1[f"Band {band//2} (MHz)"]
    dACs_er[band,:len(dAC_1)] = dAC_1[f"Band {band//2} unc (MHz)"]
    
print(dACs)
dACs = dACs[::-1,:]
# add data from dataset 2
for band in range(n_bands):
    dACs[band,len(dAC_1):] = dAC_2[f"Band {band} (MHz)"][x_ignores:]
    dACs_er[band,len(dAC_1):] = dAC_2[f"Band {band} unc (MHz)"][x_ignores:]

x_data_80 = x_data
dACs_80 = dACs
dACs_80_er = dACs_er

print(dACs_80)

[-30 -29 -28 -27 -26 -25 -24 -23 -21 -20 -19 -18 -17 -16 -15 -14 -13 -12
 -11 -10]
[[-10.95338336 -13.9006255  -17.39611648 -21.71492414 -26.2858261
  -32.34433647 -38.85824904 -46.23017499   0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [-10.95338336 -13.9006255  -17.39611648 -21.71492414 -26.2858261
  -32.34433647 -38.85824904 -46.23017499   0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [ -6.27336713  -8.18598651 -10.65166557 -13.29963187 -15.93807258
  -20.59398022 -25.66132734 -31.91111693   0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [ -6.27336713  -8.18598651 -10.65166557 -13.29963187 -15.93807258
  -20.59398022 -25.66132734 -31.91111693   0.           0.
    0.           0.           0.  

In [10]:
# Build dataset that concatenates all data taken at +60MHz intermediate detuning

x_data_60 = np.array(dAC_m20["RydDressPower (dBm)"])
x_data_lin = np.abs(np.array(dAC_m20["Measured Dressing Power (arb)"]))
x_lin_std = np.array(dAC_m20["Measured Dressing Power std (arb)"])
dACs_60 = np.zeros((n_bands,len(x_data_60)),dtype=float)
dACs_60_er = np.zeros(dACs_60.shape, dtype=float)
for band in range(n_bands):
    dACs_60[band] = dAC_m20[f"Band {band} (MHz)"]
    dACs_60_er[band] = dAC_m20[f"Band {band} unc (MHz)"]
    
x_cal = cal_fun(x_data_60)

In [11]:
# Select between +60MHz and +80MHz intermediate detuning datasets for all future plotting

dataset = 60  # 80
if dataset == 60:
    last_n = 0
    # exclude last n datapoints for fits
    if last_n > 0:
        x_data = x_cal[:-last_n] # x_data_60[:]
        dACs = dACs_60[:,:-last_n]
        dACs_er = dACs_60_er[:,:-last_n]
        # fraction of dataset excluded from fits to verify extrapolation capability of model
        x_ver = x_cal[-last_n:]#x_data_60[-5:]
        dACs_ver = dACs_60[:,-last_n:]
        dACs_ver_er = dACs_60_er[:,-last_n:]
    else:
        vn = 8
        x_data = x_cal # x_data_60[:]
        dACs = dACs_60
        dACs_er = dACs_60_er
        # fraction of dataset excluded from fits to verify extrapolation capability of model
        x_ver = x_cal[-vn:]
        dACs_ver = dACs_60[:,-vn:]
        dACs_ver_er = dACs_60_er[:,-vn:]
elif dataset == 80:
    x_data = x_data_80
    dACs = dACs_80
    dACs_er = dACs_80_er

In [12]:
dACs_ver.shape

(4, 8)

In [13]:
# Plot AC stark shift data and extracted CoG AC stark shift
fig, ax = plt.subplots(1,1)
for band in range(n_bands):
    ax.errorbar(x_data,dACs[band],yerr=dACs_er[band],label=f"Band {band}",fmt=".")
dAC_com = dACs.mean(0)
dAC_com_er = np.sqrt((dACs_er**2).sum(0))
ax.errorbar(x_data,dAC_com,yerr=dAC_com_er)
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{ac}$ (MHz)")
ax.set_title("Experimental AC stark shift data")
fig.show()

<IPython.core.display.Javascript object>

In [14]:
# PLot +60MHz AC stark shifts against measured linearized power
# Failed Attempt! Remove during refactor
fig, ax = plt.subplots(1,1)
for band in range(n_bands):
    ax.errorbar(x_data_lin,dACs[band],yerr=dACs_er[band],xerr=x_lin_std,label=f"Band {band}",fmt=".")
dAC_com = dACs.mean(0)
dAC_com_er = np.sqrt((dACs_er**2).sum(0))
ax.errorbar(x_data_lin,dAC_com,yerr=dAC_com_er)
ax.set_xlabel("Measured Dressing Power (arb)")
ax.set_ylabel("$\Delta_{ac}$ (MHz)")
ax.set_title("Experimental AC stark shift data")
#ax.set_xscale("log")
fig.show()

<IPython.core.display.Javascript object>

# Fit CoG AC stark shift to expected behavior

In [15]:
# dB data
func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)

fig, ax = plt.subplots(1,1)
#for band in range(n_bands):
#    ax.errorbar(x_data_lin,dACs[band],yerr=dACs_er[band],xerr=x_lin_std,label=f"Band {band}",fmt=".")
dAC_com = dACs.mean(0)
dAC_com_er = np.sqrt((dACs_er**2).sum(0))
guess = [-600, 0.2]

xls = np.linspace(min(x_data),max(x_data),1000)
ax.errorbar(x_data,dAC_com,yerr=dAC_com_er, label="experiment data")
ax.plot(xls, func(xls,*guess), label="fit")
ax.set_xlabel("Measured Dressing Power (arb)")
ax.set_ylabel("$\Delta_{ac}$ (MHz)")
ax.set_title("Experimental AC stark shift data")
ax.legend()
#ax.set_xscale("log")
fig.show()

<IPython.core.display.Javascript object>

In [16]:
# Use raster scan and rms of residuals to get a rough least-squares fit of data to model

def res(xdat,ydat,func,fit):
    return np.mean(ydat-func(xdat,*fit))

def res_rms(xdat,ydat,func,fit):
    return np.mean(np.sqrt(((ydat-func(xdat,*fit))**2).sum()))

# dB data
func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
# linear data
#func = lambda x, alpha, c: alpha*(np.sqrt(c**2+x**2) - c)

xdat = x_data
ydat = dAC_com
yer = dAC_com_er
alphas = -np.linspace(625,680,300)
cs = np.linspace(0.25,0.34,300)
residuals = np.zeros((len(alphas),len(cs)),dtype=float)
residuals_rms = np.zeros((len(alphas),len(cs)),dtype=float)
for i, alpha in enumerate(alphas):
    for j, c in enumerate(cs):
        residuals[i,j] = abs(res(xdat,ydat,func,[alpha,c]))
        residuals_rms[i,j] = res_rms(xdat,ydat,func,[alpha,c])

In [17]:
fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
#    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,
    max(alphas)+alphas_step/2,

]
im = ax.imshow(residuals, interpolation="none", aspect="auto", extent=extent)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
fig.show()

fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)

im = ax.imshow(residuals_rms, interpolation="none", aspect="auto", extent=extent , vmin=np.min(residuals_rms), vmax=np.min(residuals_rms)+1)
cbar = fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7, label="RMS of residuals")
cbar.set_label("RMS of fit residuals",size=12)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
ax.set_title("Fit parameter raster scan")
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("12")

fig.tight_layout()
fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
# extract optimal fitting paramters
good_inds = np.argwhere(residuals_rms == np.min(residuals_rms))[0]
fits = alphas[good_inds[0]],cs[good_inds[1]]
print(good_inds)
print(fits)

[157 141]
(-653.8795986622074, 0.29244147157190636)


In [19]:
results_dir = r"Shirley-Floquet_results\PolarizabilityPlots\RawData"
plots_dir =  r"Shirley-Floquet_results\PolarizabilityPlots"

# plot data against CoG fit
#func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
inv_cal = interp1d(y_pk, x_set, kind="linear")

guess = fits

fig,axar = plt.subplots(2,1,figsize=(5,5.5),gridspec_kw={'height_ratios': [3, 1]})#,sharex=True)

ax=axar[0]
ax.errorbar(inv_cal(xdat),ydat,yerr=yer,fmt=".",label="data")
xlin=np.linspace(min(xdat),max(xdat),1000)
ax.plot(inv_cal(xlin),func(xlin,*guess), label="fit")
#ax.set_title(f"Experiment Data Vs Fit\n$\Delta_{{ac}} = {guess[0]:.0f}*(\sqrt{{{guess[1]:.3f}^2+10^{{P/10}} }} - {guess[1]:.3f})$")
#ax.set_title(f"Experiment Data Vs Fit\n$\Delta_{{ac}} = {guess[0]:.0f}*(\sqrt{{{guess[1]:.3f}^2+P^2}} - {guess[1]:.3f})$")
#ax.set_xlabel("Dressing Power (dBm)")
#ax.set_xlabel("Dressing Power Measured (arb)")
ax.set_ylabel("$\Delta_{\\rm ac}/2\pi~\\rm (MHz)$")
ax.legend()

ax=axar[1]
ax.errorbar(inv_cal(xdat),ydat-func(xdat,*guess),yerr=yer,fmt=".")
ax.axhline(0,ls=":",c="tab:orange")
#ax.set_title("Fit Residuals")
ax.set_ylabel("Residuals (MHz)")
ax.set_xlabel("$P_{s}$ (dBm)")
#ax.set_xlabel("Dressing Power Measured (arb)")
#ax.plot(xlin,func(xlin,*guess))
axar[0].get_shared_x_axes().join(axar[0], axar[1])
axar[0].set_xticklabels([])
axar[1].set_xlim(axar[0].get_xlim())

for ax in axar:
    ax.minorticks_off()
    ax.tick_params(right=False,top=False)
    for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
        txt.set_fontsize("12")
fig.tight_layout()
fig.savefig(os.path.join(plots_dir,"UnitlessCoMFit.png"))
figs_dir = r"Z:\Public\Hybrid\Publications\Dressing in Cryostat\figures_TODO"
fig.savefig(os.path.join(figs_dir,"CoG_fit.png"))
fig.savefig(os.path.join(figs_dir,"CoG_fit.eps"))
fig.show()

<IPython.core.display.Javascript object>

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [20]:
# estimate sensitivity to a 0.1dBm miscalibration of Dressing Power
a = fits[0]
c = fits[1]
func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
dfunc = lambda x, alpha, c: -alpha*2**(x/10-2)*5**(x/10-1)*np.log(10)/np.sqrt(c**2+10**(x/10))

sens = 0.1*dfunc(-10.289,1589,0.095) # MHz

# Scan Eac to match up numerical estimates with experimental data

In [21]:
levels, basis, basis_n = None, None, None
ellipticity, scale = None, None

In [22]:
fields = {
    "Eac": np.append(np.linspace(0,15,200),np.linspace(15+75/300,90,350)),
#    "Eac": 80,
#    "ellipticity": np.linspace(0,np.sqrt(0.04),200)**2,
    "ellipticity": 0.012,
    "field_omega": tau*4780e6,
    "Edc": 0.0,
    "theta": 0
}

for key, value in fields.items():
    try:
        tmp = value[1]
    except TypeError:
        continue
    else:
        varied = (key, value)

t_level = RydStateFS(52,1,3/2)
max_det = tau*30e9
dl = 2
n_max = 2

comp = {"n_max": n_max, "dl": dl, "max_det": max_det}

if levels is None:
    levels, basis = build_basis(t_level, max_det, single_side=False, dl=dl)
    basis_n = build_n_basis(basis, n_max)
    
t_inds = [ind for ind, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]

# measured properties

# for 685nm at +60MHz wrt 4-6' transition. Generator at 181.25MHz
if dataset == 60:
    # measured resonance between nP3/2 -> nD3/2 levels
    p3o2_d3o2_res = 4839*1e6*tau
# for 685nm at +80MHz wrt 4-6' transition. Generator at 191.25MHz
elif dataset == 80:
    p3o2_d3o2_res = 4840*1e6*tau
# effective laser induced AC stark shift
laser_dAC = detuning(RydStateFS(51,2,3/2), t_level, p3o2_d3o2_res)

## Single Scan

In [23]:
#ellips = arange(0.015,0.024,0.001)
#scales = linspace(150,180,100)
H0 = build_H0(basis, t_level, offset_zeeman=1e6)
H0[t_inds,t_inds] += laser_dAC
f_inds = [(ind,state.j) for ind, state in enumerate(basis) if state.l == 2]
f_jsplit = 3e6
for ind, j in f_inds:
    H0[ind,ind] += f_jsplit*j
#for ellip in ellips: 
#    fields["ellipticity"] = ellip
try:
    del energies
    del eigenstates
except NameError:
    pass
energies, eigenstates, error = floquet_loop(basis, H0, n_max=n_max, varied=varied, energy_bands=True, **fields)

if error is not None:
    print(error)
#    basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
#    dAC_num = energies
    
#    for scale in scales:
        

240
Levels :

[
	|52,S,1/2>,
	|48,F,5/2>,
	|48,F,7/2>,
	|52,P,1/2>,
	|52,P,3/2>,
	|51,D,3/2>,
	|51,D,5/2>,
	|53,S,1/2>,
	|49,F,5/2>,
	|49,F,7/2>
]
little_d built in 0.011968135833740234
Hdc built in 0.0
Hfloquet put together in 0.0
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 0.0, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.006981372833251953s
Normalization Complete after 0.002992391586303711 s
floquet_diag call completed in 0.02393627166748047s
Diagonalization complete for independent variable entry, value 0, 0.0
finding good eigenvectors for |level, n> = ||52,S,1/2>,-2>
finding good eigenvectors for |level, n> = ||52,S,1/2>,-1>
finding good eigenvectors for |level, n> = ||52,S,1/2>,0>
finding good eigenvectors for |level, n> = ||52,S,1/2>,1>
finding good eigenvectors for |level, n> = ||52,S,1/2>,2>
finding good eigenvectors for |level

little_d built in 0.011967658996582031
Hdc built in 0.0
Hfloquet put together in 0.0
Floquet matrix built after 0.012965917587280273 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 0.6783919597989949, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.1326448917388916s
Normalization Complete after 0.0029914379119873047 s
floquet_diag call completed in 0.14860224723815918s
Diagonalization complete for independent variable entry, value 9, 0.6783919597989949
re-arrangement done after time 0.004987001419067383s
little_d built in 0.011967897415161133
Hdc built in 0.0009975433349609375
Hfloquet put together in 0.0
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 0.7537688442211055, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13364243507385254s
Normaliza

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 1.658291457286432, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13181757926940918s
Normalization Complete after 0.003989219665527344 s
floquet_diag call completed in 0.1487724781036377s
Diagonalization complete for independent variable entry, value 22, 1.658291457286432
re-arrangement done after time 0.005984067916870117s
little_d built in 0.011968135833740234
Hdc built in 0.0
Hfloquet put together in 0.000997304916381836
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 1.7336683417085426, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13524222373962402s
Normalization Complete after 0.0029914379119873047 s
floquet_diag call completed in 0.15219640731811523s
Diagonalization complete for independent variable entry, value

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 2.7135678391959797, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13663458824157715s
Normalization Complete after 0.001994609832763672 s
floquet_diag call completed in 0.1545870304107666s
Diagonalization complete for independent variable entry, value 36, 2.7135678391959797
re-arrangement done after time 0.005983829498291016s
little_d built in 0.011968374252319336
Hdc built in 0.0
Hfloquet put together in 0.0
Floquet matrix built after 0.013962745666503906 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 2.78894472361809, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13065099716186523s
Normalization Complete after 0.003989219665527344 s
floquet_diag call completed in 0.14960050582885742s
Diagonalization complete for independent variable entry, value 37, 2.7889447236

Hdc built in 0.0
Hfloquet put together in 0.000997304916381836
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 3.844221105527638, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.17502188682556152s
Normalization Complete after 0.004986763000488281 s
floquet_diag call completed in 0.19297409057617188s
Diagonalization complete for independent variable entry, value 51, 3.844221105527638
re-arrangement done after time 0.0069811344146728516s
little_d built in 0.014960289001464844
Hdc built in 0.0
Hfloquet put together in 0.000997304916381836
Floquet matrix built after 0.01595759391784668 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 3.9195979899497484, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13897323608398438s
Normalization Complete after 0.00

...

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 76.93481375358166, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.16679954528808594s
Normalization Complete after 0.002992391586303711 s
floquet_diag call completed in 0.18275737762451172s
Diagonalization complete for independent variable entry, value 488, 76.93481375358166
re-arrangement done after time 0.006981372833251953s
little_d built in 0.012965202331542969
Hdc built in 0.0
Hfloquet put together in 0.0009975433349609375
Floquet matrix built after 0.013962745666503906 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 77.14899713467048, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.14495515823364258s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.161909818649292s
Diagonalization complete for independent variable entry, value

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 80.14756446991403, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13962674140930176s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.15558433532714844s
Diagonalization complete for independent variable entry, value 503, 80.14756446991403
re-arrangement done after time 0.00498652458190918s
little_d built in 0.011968135833740234
Hdc built in 0.000997304916381836
Hfloquet put together in 0.0
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 80.36174785100286, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13863015174865723s
Normalization Complete after 0.002991199493408203 s
floquet_diag call completed in 0.15558409690856934s
Diagonalization complete for independent variable entry, value 

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 83.1461318051576, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13776373863220215s
Normalization Complete after 0.003989219665527344 s
floquet_diag call completed in 0.1557157039642334s
Diagonalization complete for independent variable entry, value 517, 83.1461318051576
re-arrangement done after time 0.004986763000488281s
little_d built in 0.013962507247924805
Hdc built in 0.0
Hfloquet put together in 0.0009975433349609375
Floquet matrix built after 0.014960050582885742 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 83.36031518624641, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13364291191101074s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.151594877243042s
Diagonalization complete for independent variable entry, value 51

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 86.14469914040114, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.14076685905456543s
Normalization Complete after 0.0029921531677246094 s
floquet_diag call completed in 0.156724214553833s
Diagonalization complete for independent variable entry, value 531, 86.14469914040114
re-arrangement done after time 0.005983829498291016s
little_d built in 0.011968135833740234
Hdc built in 0.0
Hfloquet put together in 0.000997304916381836
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 86.35888252148997, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.1406240463256836s
Normalization Complete after 0.0029916763305664062 s
floquet_diag call completed in 0.15658116340637207s
Diagonalization complete for independent variable entry, value 

little_d built in 0.012068033218383789
Hdc built in 0.0009980201721191406
Hfloquet put together in 0.0
Floquet matrix built after 0.01306605339050293 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 89.35744985673352, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.14487433433532715s
Normalization Complete after 0.0029916763305664062 s
floquet_diag call completed in 0.16192889213562012s
Diagonalization complete for independent variable entry, value 546, 89.35744985673352
re-arrangement done after time 0.004986763000488281s
little_d built in 0.011138916015625
Hdc built in 0.0
Hfloquet put together in 0.0
Floquet matrix built after 0.013133764266967773 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 89.57163323782234, ellipticity 0.012, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13463997840881348s
Normalizatio

## Iterative scans
Repeat raster-scan to least-squares fit technique from CoG fit above

In [24]:
ellips = np.arange(0.01125,0.0125,0.00025)
scales = np.arange(63,65,0.01)
#scales=[167]
residuals = np.zeros((len(ellips),len(scales)), dtype=float)
residuals_rms = np.zeros(residuals.shape, dtype=float)
H0 = build_H0(basis, t_level, offset_zeeman=1e6)
H0[t_inds,t_inds] += laser_dAC
f_inds = [(ind,state.j) for ind, state in enumerate(basis) if state.l == 2]
f_jsplit = 3e6
for ind, j in f_inds:
    H0[ind,ind] += f_jsplit*j

basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
#Reorder experiment bands
tmp = basis_t[1]
basis_t[1] = basis_t[2]
basis_t[2] = tmp
basis_t = basis_t[1:] + basis_t[:1]
basis_t = basis_t[::-1]

for i, ellip in enumerate(ellips):
    try:
        del energies
        del eigenstates
    except NameError:
        pass
    fields["ellipticity"] = ellip
    energies, eigenstates = floquet_loop(basis, H0, n_max=n_max, varied=varied, energy_bands=True, **fields)

    dAC_num = energies
    
    for j, scale in enumerate(scales):
        x_ac = 20*np.log10(varied[1]/scale)
        #fig, axar = plt.subplots(1,2,figsize=(12,6))
        #ax=axar[0]
        #ax1=axar[1]
        for band in range(n_bands):
            y_num = 1e-6*(energies[basis_t[::-1][band]]-energies[basis_t[::-1][band],0])/tau
            y_ex = dACs[band]
            dy_er = dACs_er[band]
            #ax.plot(x_ac,y_num,label=f"numerical band {band}")
            #ax.errorbar(x_data,y_ex,yerr=dy_er,fmt=".",label=f"experimental band {band}")
            y_int = np.interp(x_data,x_ac,y_num)
            res = y_ex-y_int
            #ax1.errorbar(x_data,res,yerr=dy_er,fmt=".",label=f"Band {band}")
            residuals[i,j] += res.mean()
            residuals_rms[i,j] = np.sqrt(residuals_rms[i,j]**2+(res**2).sum())
        '''ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot 4780MHz$\n$\epsilon={ellip:.3f}; V_{{ref}} = {scale:.0f}V/m$")
        ax.set_xlabel("Dressing Power (dBm)")
        ax.set_ylabel("$\Delta_{ac}-CoM (MHz)$")
        ax.set_xlim(-31,-9)
        ax.legend(loc=3)
        ax1.set_title("Fit residuals")
        ax1.set_xlabel("Dressing Power (dBm)")
        ax1.set_ylabel("Fit Residuals (MHz)")
        fig.show'''

240
Levels :

[
	|52,S,1/2>,
	|48,F,5/2>,
	|48,F,7/2>,
	|52,P,1/2>,
	|52,P,3/2>,
	|51,D,3/2>,
	|51,D,5/2>,
	|53,S,1/2>,
	|49,F,5/2>,
	|49,F,7/2>
]
little_d built in 0.013962507247924805
Hdc built in 0.0
Hfloquet put together in 0.0009975433349609375
Floquet matrix built after 0.014960050582885742 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 0.0, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.005984067916870117s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.023936033248901367s
Diagonalization complete for independent variable entry, value 0, 0.0
finding good eigenvectors for |level, n> = ||52,S,1/2>,-2>
finding good eigenvectors for |level, n> = ||52,S,1/2>,-1>
finding good eigenvectors for |level, n> = ||52,S,1/2>,0>
finding good eigenvectors for |level, n> = ||52,S,1/2>,1>
finding good eigenvectors for |level, n> = ||52,S,1/2>,2>
finding good e

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 0.6783919597989949, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13065099716186523s
Normalization Complete after 0.003989219665527344 s
floquet_diag call completed in 0.14960002899169922s
Diagonalization complete for independent variable entry, value 9, 0.6783919597989949
re-arrangement done after time 0.00498652458190918s
little_d built in 0.011968135833740234
Hdc built in 0.0
Hfloquet put together in 0.0
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 0.7537688442211055, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.1326453685760498s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.14960002899169922s
Diagonalization complete for independent variable entry, value 10, 0.7537688

little_d built in 0.012965202331542969
Hdc built in 0.0
Hfloquet put together in 0.000997781753540039
Floquet matrix built after 0.014960289001464844 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 1.7336683417085426, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13353753089904785s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.1514897346496582s
Diagonalization complete for independent variable entry, value 23, 1.7336683417085426
re-arrangement done after time 0.00498652458190918s
little_d built in 0.011968135833740234
Hdc built in 0.0
Hfloquet put together in 0.0
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 1.809045226130653, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13164830207824707s
Normaliz

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 2.7135678391959797, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13164806365966797s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.14960002899169922s
Diagonalization complete for independent variable entry, value 36, 2.7135678391959797
re-arrangement done after time 0.005984067916870117s
little_d built in 0.011967897415161133
Hdc built in 0.0
Hfloquet put together in 0.0009975433349609375
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 2.78894472361809, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13175463676452637s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.14771199226379395s
Diagonalization complete for independent variable entry, 

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 3.844221105527638, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13578319549560547s
Normalization Complete after 0.0019943714141845703 s
floquet_diag call completed in 0.15373516082763672s
Diagonalization complete for independent variable entry, value 51, 3.844221105527638
re-arrangement done after time 0.004987001419067383s
little_d built in 0.011967658996582031
Hdc built in 0.0
Hfloquet put together in 0.0009975433349609375
Floquet matrix built after 0.012965202331542969 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 3.9195979899497484, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.1326453685760498s
Normalization Complete after 0.003989219665527344 s
floquet_diag call completed in 0.14959979057312012s
Diagonalization complete for independent variable entry,

...

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 79.93338108882521, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13663458824157715s
Normalization Complete after 0.001995086669921875 s
floquet_diag call completed in 0.15358948707580566s
Diagonalization complete for independent variable entry, value 502, 79.93338108882521
re-arrangement done after time 0.004987001419067383s
little_d built in 0.010970592498779297
Hdc built in 0.0
Hfloquet put together in 0.0
Floquet matrix built after 0.012965679168701172 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 80.14756446991403, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13364243507385254s
Normalization Complete after 0.0029921531677246094 s
floquet_diag call completed in 0.15059757232666016s
Diagonalization complete for independent variable entry, value 503, 80.14

little_d built in 0.010970592498779297
Hdc built in 0.0
Hfloquet put together in 0.0009970664978027344
Floquet matrix built after 0.012964963912963867 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 83.1461318051576, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13933086395263672s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.1552877426147461s
Diagonalization complete for independent variable entry, value 517, 83.1461318051576
re-arrangement done after time 0.005984067916870117s
little_d built in 0.011967897415161133
Hdc built in 0.0
Hfloquet put together in 0.0
Floquet matrix built after 0.012965679168701172 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 83.36031518624641, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13663434982299805s
Normaliz

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 85.93051575931231, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13774800300598145s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.15370535850524902s
Diagonalization complete for independent variable entry, value 530, 85.93051575931231
re-arrangement done after time 0.005984306335449219s
little_d built in 0.011967897415161133
Hdc built in 0.0
Hfloquet put together in 0.000997781753540039
Floquet matrix built after 0.012965679168701172 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 86.14469914040114, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.13763165473937988s
Normalization Complete after 0.002991914749145508 s
floquet_diag call completed in 0.15358924865722656s
Diagonalization complete for independent variable entry, 

Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 88.92908309455586, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.14162111282348633s
Normalization Complete after 0.0029921531677246094 s
floquet_diag call completed in 0.15857601165771484s
Diagonalization complete for independent variable entry, value 544, 88.92908309455586
re-arrangement done after time 0.005984067916870117s
little_d built in 0.011967897415161133
Hdc built in 0.0
Hfloquet put together in 0.0009975433349609375
Floquet matrix built after 0.01296544075012207 s
Begin Diagonalizing
Diagonalized Hfloquet for with dims (240, 240)
AC parameters Eac 89.1432664756447, ellipticity 0.01125, field_omega 30033625768.31842
DC parameters Edc 0.0, theta 0
Diagonalized after time: 0.1371757984161377s
Normalization Complete after 0.0029921531677246094 s
floquet_diag call completed in 0.15313339233398438s
Diagonalization complete for independent variable entry, 

ValueError: too many values to unpack (expected 2)

In [ ]:
fig,ax = plt.subplots(1,1)
eps_step = (max(ellips)-min(ellips))/len(ellips)
scale_step = (max(scales)-min(scales))/len(scales)
extent = [
    min(scales)-scale_step/2,
    max(scales)+scale_step/2,
    max(ellips)+eps_step/2,
    min(ellips)-eps_step/2,

]
im = ax.imshow(residuals, interpolation="none", aspect="auto", extent=extent)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("$V_{ref}$")
ax.set_ylabel("$\epsilon$")
ax.set_xticks(scales.astype(float),minor=2)
ax.set_yticks(ellips.astype(float),minor=2)
fig.show()

fig,ax = plt.subplots(1,1)
im = ax.imshow(residuals_rms, interpolation="none", aspect="auto", extent=extent, vmin=np.min(residuals_rms), vmax=np.min(residuals_rms)+1)
cbar = fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7, label="RMS of residuals")
cbar.set_label("RMS of fit residuals",size=12)
ax.set_xlabel("$V_{ref}$")
ax.set_ylabel("$\epsilon$")
ax.set_xticks(scales.astype(float),minor=2)
ax.set_yticks(ellips.astype(float),minor=2)
ax.set_title("Fit parameter raster scan")
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("12")

fig.tight_layout()
fig.show()

In [ ]:
# Extract fit parameters
good_inds = np.argwhere(residuals_rms == np.min(residuals_rms))[0]
ellipticity = float(ellips[good_inds[0]])
scale = float(scales[good_inds[1]])
print(np.argwhere(residuals_rms == np.min(residuals_rms)))
print(ellipticity, scale)
# delete large residuals arrays to make space for final scan
del residuals
del residuals_rms

In [ ]:
# manually enter optimal fit values if known, to avoid running raster scan each time
if dataset == 60:
    # ellipticity = 0.0135
    if ellipticity is None:
        ellipticity = 0.0120
    if scale is None:
        scale = 64.1  # V/m @ 0dBm
elif dataset == 80:
    if ellipticity is None:
        ellipticity = 0.0245
    if scale is None:
        scale = 167.822  # V/m @ 0dBm
fields = {
    "Eac": np.append(np.linspace(0,15,200),np.linspace(15+75/300,90,350)),
#    "Eac": 80,
#    "ellipticity": np.linspace(0,np.sqrt(0.04),200)**2,
    "ellipticity": ellipticity,
    "field_omega": tau*4780e6,
    "Edc": 0.0,
    "theta": 0
}

for key, value in fields.items():
    try:
        tmp = value[1]
    except TypeError:
        continue
    else:
        varied = (key, value)

t_level = RydStateFS(52,1,3/2)
max_det = tau*30e9
dl = 2
n_max = 2

comp = {"n_max": n_max, "dl": dl, "max_det": max_det}

if levels is None:
    levels, basis = build_basis(t_level, max_det, single_side=False, dl=dl)
    basis_n = build_n_basis(basis, n_max)

## Single Scan

In [ ]:
# Re-run 1D scan using optimized fit paramters
try:
    del energies
    del eigenstates
except NameError:
    pass

#ellips = arange(0.015,0.024,0.001)
#scales = linspace(150,180,100)
H0 = build_H0(basis, t_level, offset_zeeman=1e6)
H0[t_inds,t_inds] += laser_dAC
f_inds = [(ind,state.j) for ind, state in enumerate(basis) if state.l == 2]
f_jsplit = 3e6
for ind, j in f_inds:
    H0[ind,ind] += f_jsplit*j

#for ellip in ellips:
#    fields["ellipticity"] = ellip
energies, eigenstates, error = floquet_loop(basis, H0, n_max=n_max, varied=varied, energy_bands=True, **fields)

if error is not None:
    print()
    print(error)
    
#    basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
#    dAC_num = energies
    
#    for scale in scales:
        

In [ ]:
# create plot of experiment data and fitted numerical data

inv_cal = interp1d(y_pk, x_set, kind="linear")
colors = ['tab:blue','tab:orange', 'tab:green', 'tab:red']

ax_fontsize = 13
tick_labelsize = 11
legend_fontsize = 11
markersize = 3.5
markers = "o^vsD*"
linestyles = ['--','-.', ':', '-']

fig,axar = plt.subplots(2,1,figsize=(4.5,5.5),gridspec_kw={'height_ratios': [3, 1]})#,sharex=True)
ax = axar[0]
for band in range(n_bands):
    lab = f"experiment" if band == 0 else ""
    lab = f"band {band+1}"
    #lab = f"experimental {band}"
    ax.errorbar(
        inv_cal(x_data)[:-vn],
        (dACs[band]-func(x_data,*fits))[:-vn],
        yerr = dACs_er[band][:-vn],
        fmt = markers[band], 
        label=lab,
        c=colors[band],
        ms=markersize
    )
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)

#y = 0.6
#x_data_3_p = x_data_3+np.log10(1+y*10**-(x_data_3/20))
#for band in range(n_bands):
#    lab = "verification data" if band == 0 else ""
#    ax.errorbar(x_data_3_p,dACs3[band]-func(x_data_3_p,*fits)+10*0-22, yerr = dACs3_er[band], fmt="o", label=lab)

if dataset == 60:
    x_data_2 = x_ver
    dACs_2 = dACs_ver
    dACs_2_er = dACs_ver_er
elif dataset == 80:
    x_data_2 = x_data_60
    dACs_2 = dACs_60
    dACs_2_er = dACs_60_er
ms=8
plot_vers = True
if plot_vers:
    y = 0.0
    lab=""
    #x_data_2_p = x_data_2+np.log10(1+y*10**-(x_data_2/20))
    for band in range(n_bands):
        # lab = "verification data" if band == 0 else ""
        ax.errorbar(
            inv_cal(x_data_2),
            dACs_2[band]-func(x_data_2,*fits),
            yerr = dACs_2_er[band], 
            fmt=markers[band],
            label=lab,
            fillstyle="none",
            c=colors[band],
            ms=markersize
        )

    
thrs = 0.10


basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]

#Reorder experiment bands
tmp = basis_t[1]
basis_t[1] = basis_t[2]
basis_t[2] = tmp
basis_t = basis_t[1:] + basis_t[:1]

interesting_inds = []
for t_ind in basis_t:
    for k, ps in enumerate(eigenstates[:, t_ind, :]):
        if max(np.absolute(ps)**2) > thrs:
            interesting_inds.append(k)

interesting_inds = list(set(interesting_inds))
x_ac = 20*np.log10(varied[1]/scale)
interp_inds = np.argwhere((x_ac > min(y_pk))*(x_ac < max(y_pk))).T[0]
x_acp = x_ac[interp_inds]
energiesp = energies[:,interp_inds]
for i,k in enumerate(basis_t):
#for i,k in enumerate(interesting_inds[::-1]): 
    lab = f"fit" if i==0 else ""
    #lab = f"fit {i}"
    print(k)
    print(colors[i])
    ax.plot(inv_cal(x_acp), 1e-6*(energiesp[k]-energies[k,0])/tau-func(x_acp,*fits), ls=linestyles[i], label=lab, c=colors[i])
#ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
#ax.axvline(inv_cal(20*np.log10(25.5/scale)),ls=":",label="Expected Polarizability minumum")
zero_xs = [78.6,82.6,85.6,89.6]
for i,zerox in enumerate(zero_xs):
    ax.axvline(inv_cal(20*np.log10(zerox/scale)),ls="--",c=colors[i],linewidth=1)#,label="Polarizability minumum")
    print(f"zero_x : {inv_cal(20*np.log10(zerox/scale))}dBm")

#ax.axvline()

ax.set_xlim(-30.5,-5.25)
ax.set_ylim(-88,66)

ax.legend(loc=3)
#ax.set_xlabel("Calibrated Dressing Power (dBm)")
#ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC}$ - CoG (MHz)")
ellip = fields["ellipticity"]
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
#ax.set_title(f"AC stark shift fit - CoG\n$\omega_{{d}}=2\pi \cdot 4780MHz$\n$\epsilon={ellip:.5f}; V_{{ref}} = {scale:.2f}V/m$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize(ax_fontsize)

ax = axar[1]
for band in range(n_bands):
    y_interp = np.interp(x_data,x_ac, 1e-6*(energies[basis_t[band]]-energies[basis_t[band],0])/tau)
    '''
    if band == 1:
        band = 2
    elif band == 2:
        band = 1
    '''
    res = dACs[band] - y_interp
    ax.errorbar(inv_cal(x_data)[:-vn],res[:-vn],yerr=dACs_er[band][:-vn],fmt=markers[band],label=f"band {band}", c=colors[band],ms=markersize)
    if plot_vers:
        y_interp = np.interp(x_data_2,x_ac,1e-6*(energies[basis_t[band]]-energies[basis_t[band],0])/tau)
        lab = "verification data" if band == 0 else ""
        ax.errorbar(inv_cal(x_data_2),dACs_2[band]-y_interp, yerr = dACs_2_er[band], fmt=markers[band], label=lab, fillstyle="none", c=colors[band],markersize=markersize)
ax.set_ylim(-6,6)
ax.set_xlim(-30.5,-5.5)
#ax.legend(loc=3)
ax.axhline(0,ls=":",c="r")

#ax.set_xlabel("Calibrated Dressing Power (dBm)")

ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("Residuals (MHz)")
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

axar[0].get_shared_x_axes().join(axar[0], axar[1])
axar[0].set_xticklabels([])
axar[1].set_xlim(axar[0].get_xlim())
#axar[0].set_xlim
#ax.set_title(f"AC stark shift fit residuals\n$\omega_{{d}}=2\pi \cdot 4780MHz$\n$\epsilon={ellip:.5f}; V_{{ref}} = {scale:.2f}V_{{cal}}/m$")

for ax in axar:
    ax.tick_params(labelsize=tick_labelsize)
    for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
        txt.set_fontsize(ax_fontsize)

fig.tight_layout()
fig.show()
fig.savefig(os.path.join(plots_dir,"Calibrated AC stark map.png"))
fig.savefig(os.path.join(plots_dir,"Calibrated AC stark map.eps"))

In [ ]:
# create plot of experiment data and fitted numerical data

inv_cal = interp1d(y_pk, x_set, kind="linear")
colors = ['tab:blue','tab:orange', 'tab:green', 'tab:red']

ax_fontsize = 13
tick_labelsize = 11
legend_fontsize = 11
markersize = 3.5
markers = "o^vsD*"
linestyles = ['--','-.', ':', '-']

fig,axar = plt.subplots(2,1,figsize=(4.5,5.5),gridspec_kw={'height_ratios': [3, 1]})#,sharex=True)
ax = axar[0]
for band in range(n_bands):
    lab = f"experiment" if band == 0 else ""
    lab = f"band {band+1}"
    #lab = f"experimental {band}"
    ax.errorbar(
        inv_cal(x_data)[:-vn],
        (dACs[band]-func(x_data,*fits))[:-vn],
        yerr = dACs_er[band][:-vn],
        fmt = markers[band], 
        label=lab,
        c=colors[band],
        ms=markersize
    )
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)

#y = 0.6
#x_data_3_p = x_data_3+np.log10(1+y*10**-(x_data_3/20))
#for band in range(n_bands):
#    lab = "verification data" if band == 0 else ""
#    ax.errorbar(x_data_3_p,dACs3[band]-func(x_data_3_p,*fits)+10*0-22, yerr = dACs3_er[band], fmt="o", label=lab)

if dataset == 60:
    x_data_2 = x_ver
    dACs_2 = dACs_ver
    dACs_2_er = dACs_ver_er
elif dataset == 80:
    x_data_2 = x_data_60
    dACs_2 = dACs_60
    dACs_2_er = dACs_60_er
ms=8
plot_vers = True
if plot_vers:
    y = 0.0
    lab=""
    #x_data_2_p = x_data_2+np.log10(1+y*10**-(x_data_2/20))
    for band in range(n_bands):
        # lab = "verification data" if band == 0 else ""
        ax.errorbar(
            inv_cal(x_data_2),
            dACs_2[band]-func(x_data_2,*fits),
            yerr = dACs_2_er[band], 
            fmt=markers[band],
            label=lab,
            fillstyle="none",
            c=colors[band],
            ms=markersize
        )

    
thrs = 0.10


basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]

#Reorder experiment bands
tmp = basis_t[1]
basis_t[1] = basis_t[2]
basis_t[2] = tmp
basis_t = basis_t[1:] + basis_t[:1]

interesting_inds = []
for t_ind in basis_t:
    for k, ps in enumerate(eigenstates[:, t_ind, :]):
        if max(np.absolute(ps)**2) > thrs:
            interesting_inds.append(k)

interesting_inds = list(set(interesting_inds))
x_ac = 20*np.log10(varied[1]/scale)
interp_inds = np.argwhere((x_ac > min(y_pk))*(x_ac < max(y_pk))).T[0]
x_acp = x_ac[interp_inds]
energiesp = energies[:,interp_inds]
for i,k in enumerate(basis_t):
#for i,k in enumerate(interesting_inds[::-1]): 
    lab = f"fit" if i==0 else ""
    #lab = f"fit {i}"
    print(k)
    print(colors[i])
    ax.plot(inv_cal(x_acp), 1e-6*(energiesp[k]-energies[k,0])/tau-func(x_acp,*fits), ls=linestyles[i], label=lab, c=colors[i])
#ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
#ax.axvline(inv_cal(20*np.log10(25.5/scale)),ls=":",label="Expected Polarizability minumum")
zero_xs = [78.6,82.6,85.6,89.6]
for i,zerox in enumerate(zero_xs):
    ax.axvline(inv_cal(20*np.log10(zerox/scale)),ls="--",c=colors[i],linewidth=1)#,label="Polarizability minumum")
    print(f"zero_x : {inv_cal(20*np.log10(zerox/scale))}dBm")

#ax.axvline()

ax.set_xlim(-30.5,-5.25)
ax.set_ylim(-88,66)

ax.legend(loc=3)
#ax.set_xlabel("Calibrated Dressing Power (dBm)")
#ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC}$ - CoG (MHz)")
ellip = fields["ellipticity"]
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
#ax.set_title(f"AC stark shift fit - CoG\n$\omega_{{d}}=2\pi \cdot 4780MHz$\n$\epsilon={ellip:.5f}; V_{{ref}} = {scale:.2f}V/m$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize(ax_fontsize)

ax = axar[1]
for band in range(n_bands):
    y_interp = np.interp(x_data,x_ac, 1e-6*(energies[basis_t[band]]-energies[basis_t[band],0])/tau)
    '''
    if band == 1:
        band = 2
    elif band == 2:
        band = 1
    '''
    res = dACs[band] - y_interp
    ax.errorbar(inv_cal(x_data)[:-vn],res[:-vn],yerr=dACs_er[band][:-vn],fmt=markers[band],label=f"band {band}", c=colors[band],ms=markersize)
    if plot_vers:
        y_interp = np.interp(x_data_2,x_ac,1e-6*(energies[basis_t[band]]-energies[basis_t[band],0])/tau)
        lab = "verification data" if band == 0 else ""
        ax.errorbar(inv_cal(x_data_2),dACs_2[band]-y_interp, yerr = dACs_2_er[band], fmt=markers[band], label=lab, fillstyle="none", c=colors[band],markersize=markersize)
ax.set_ylim(-6,6)
ax.set_xlim(-30.5,-5.5)
#ax.legend(loc=3)
ax.axhline(0,ls=":",c="r")

#ax.set_xlabel("Calibrated Dressing Power (dBm)")

ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("Residuals (MHz)")
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

axar[0].get_shared_x_axes().join(axar[0], axar[1])
axar[0].set_xticklabels([])
axar[1].set_xlim(axar[0].get_xlim())
#axar[0].set_xlim
#ax.set_title(f"AC stark shift fit residuals\n$\omega_{{d}}=2\pi \cdot 4780MHz$\n$\epsilon={ellip:.5f}; V_{{ref}} = {scale:.2f}V_{{cal}}/m$")

for ax in axar:
    ax.tick_params(labelsize=tick_labelsize)
    for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
        txt.set_fontsize(ax_fontsize)

fig.tight_layout()
fig.show()
fig.savefig(os.path.join(plots_dir,"Calibrated AC stark map.png"))
fig.savefig(os.path.join(plots_dir,"Calibrated AC stark map.eps"))

# Legacy
## ALL CODE BELOW THIS POINT IS NOT IN ACTIVE USE, KEPT AROUND FOR REMINDERS OF SYNTAX AND DEBUGGING

In [ ]:
thrs = 0.5
fig, ax = plt.subplots(1,1, figsize=(5,5))

basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
interesting_inds = []
for t_ind in basis_t:
    for k, ps in enumerate(eigenstates[:, t_ind, :]):
        if max(np.absolute(ps)**2) > thrs:
            interesting_inds.append(k)

interesting_inds = list(set(interesting_inds))
scale = 79.5/0.474
x_ac = 20*np.log10(varied[1]/scale)
for k in interesting_inds:
    ax.plot(x_ac, 1e-6*np.real(energies[k,:]-energies[k,0]*0)/tau,label = basis[k%len(basis)].ket())

scale = 79.5/0.474
x_ac = 20*np.log10(varied[1]/scale)
for band in range(n_bands):
    ax.errorbar(x_data, dACs[band], yerr = dACs_er[band], fmt = ".", label=f"band {band}")

ax.set_title(f"AC stark shifts\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$; $\epsilon = {fields['ellipticity']:.3f}$; $P_{{scale}}={scale:.0f}V/m$")
ax.set_ylabel("$\Delta_{AC}(MHz)$")
ax.set_xlabel("Dressing Power (dBm)")
#ax.set_ylim(-900,-80)
ax.set_xlim(-20,max(x_ac))
#ax.set_xscale("log")
#ax.set_yscale("symlog")
#ax.set_title(f"Bands wrt Ellipticity\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$\nEac = {fields['Eac']:.3f} V/m\n")
#ax.set_ylabel(f"Band Energy - $E_{{{t_level.ket()}}}$ (MHz)")
#ax.set_xlabel("Dressing Field Ellipticity")
ax.legend()
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()
#fig.savefig(f"Theory-Experiment_handscale_ellipticity_{fields['ellipticity']:.3f}.png")#
#fig.savefig(f"Band Frequency ellipticity.png")

In [ ]:

fits = [alphas[42],cs[67]]
func = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
xdat = x_data_dBm[1:]

fig,ax = plt.subplots(1,1)
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".", label=f"band {band}")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
ax.legend()
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC}$ at 1190 (MHz)")
ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func2(xdat,*fits),yerr=yer,fmt=".")
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("Mean AC stark shift - Fit value (MHz)")
ax.set_title(f"Fit Residuals\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()

fig.show()

print(fits)

In [ ]:
20*np.log10(55/scale)

In [ ]:
dACs.mean[1]

def abs_split(ar):
    return abs(max(ar)-min(ar))
dAC_CoM = (np.array([np.nanmean([row[y_str.format(band)] for band in bands]) for ind,row in res_df.iterrows()])-bare_res)
dAC_CoM_er = np.array([np.sum([row[er_str.format(band)]**2 for band in bands]) for ind,row in res_df.iterrows()])
dAC_split = np.array([abs_split([row[y_str.format(band)] for band in bands]) for ind,row in res_df.iterrows()])

In [ ]:
fig, ax = plt.subplots(1,1)
ax.errorbar(x_data_dBm[1:],dAC_CoM[1:],yerr=dAC_CoM_er[1:],fmt=".")
ax.plot(x_data_dBm[1:],dAC_CoM[1:]+dAC_split[1:]/2)
ax.plot(x_data_dBm[1:],dAC_CoM[1:]-dAC_split[1:]/2)
ax.set_title("COM and Splitting verify")
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".")
fig.show()

In [ ]:
res_df = pd.read_csv("Measured_RF_Resonances_4780.csv")
#res_df = pd.read_csv("Measured_RF_Resonances_4780_2.csv")
res_df

In [ ]:
bands = range(1,6)
x_data_dBm = res_df["RF power (dBm)"]
x_data_Vs = np.sqrt(10**(x_data_dBm/10))
y_str = "mu {} (MHz)"
er_str = "sig {} (MHz)"
bare_res = res_df["mu 1 (MHz)"][0]
fig, ax = plt.subplots(1,1, figsize=(5,5))
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = 2*(measured_res-bare_res)
    dAC_er = 2*errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".")
ax.set_yscale("symlog")
ax.set_title("Measured AC stark shifts\n4780MHz Dressing field")
ax.set_xlabel("Drive Power (dBm)")
ax.set_ylabel("$-\Delta_{AC} (MHz)$")
fig.tight_layout()
fig.show()
# fig.savefig("Unscaled Wiki AC stark shifts.png")

In [ ]:
def abs_split(ar):
    return abs(max(ar)-min(ar))
dAC_CoM = (np.array([np.nanmean([row[y_str.format(band)] for band in bands]) for ind,row in res_df.iterrows()])-bare_res)
dAC_CoM_er = np.array([np.sum([row[er_str.format(band)]**2 for band in bands]) for ind,row in res_df.iterrows()])
dAC_split = np.array([abs_split([row[y_str.format(band)] for band in bands]) for ind,row in res_df.iterrows()])

In [ ]:
fig, ax = plt.subplots(1,1)
ax.errorbar(x_data_dBm[1:],dAC_CoM[1:],yerr=dAC_CoM_er[1:],fmt=".")
ax.plot(x_data_dBm[1:],dAC_CoM[1:]+dAC_split[1:]/2)
ax.plot(x_data_dBm[1:],dAC_CoM[1:]-dAC_split[1:]/2)
ax.set_title("COM and Splitting verify")
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".")
fig.show()

In [ ]:
xdat = x_data_Vs[1:]
ydat = dAC_CoM[1:]
yer = dAC_CoM_er[1:]

func = lambda x, alpha, c: alpha*(np.sqrt(c**2+x**2)-c)
guess= [-1.76e3/2,1.4e-1]

popt,pcov = curve_fit(func,xdat,ydat,sigma=yer,p0=guess)
perr = np.sqrt(np.diag(pcov))

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat,yerr=yer,fmt=".")
ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func(xlin,*popt))
ax.plot(xlin,func(xlin,*guess))
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func(xdat,*guess),yerr=yer,fmt=".")
fig.show()

In [ ]:
def res(xdat,ydat,func,fit):
    return np.mean(ydat-func(xdat,*fit))

def res_rms(xdat,ydat,func,fit):
    return np.mean(np.sqrt(((ydat-func(xdat,*fit))**2).sum()))

alphas = np.linspace(-1.825,-1.55,100)*1e3/2
cs = np.linspace(0.8,1.3,100)*1e-1
residuals = np.zeros((len(alphas),len(cs)),dtype=float)
residuals_rms = np.zeros((len(alphas),len(cs)),dtype=float)
for i, alpha in enumerate(alphas):
    for j, c in enumerate(cs):
        residuals[i,j] = abs(res(xdat,ydat,func,[alpha,c]))
        residuals_rms[i,j] = res_rms(xdat,ydat,func,[alpha,c])

In [ ]:
fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,

]
im = ax.imshow(residuals, interpolation="none", aspect="auto", extent=extent)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
fig.show()

fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,

]
im = ax.imshow(residuals_rms, interpolation="none", aspect="auto", extent=extent, vmin=np.min(residuals_rms), vmax=np.min(residuals_rms)+10)
cbar = fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7, label="RMS of residuals")
cbar.set_label("RMS of fit residuals",size=12)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
ax.set_title("Fit parameter raster scan")
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("12")

fig.tight_layout()
fig.show()

In [ ]:
np.argwhere(residuals_rms==np.min(residuals_rms))[0]

In [ ]:
fits = [alphas[42],cs[67]]
func2 = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)
xdat = x_data_dBm[1:]

fig,ax = plt.subplots(1,1)
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    ax.errorbar(x_data_dBm[1:], dAC[1:], yerr = dAC_er[1:], fmt = ".", label=f"band {band}")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
ax.legend()
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC}$ at 1190 (MHz)")
ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func2(xdat,*fits),yerr=yer,fmt=".")
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("Mean AC stark shift - Fit value (MHz)")
ax.set_title(f"Fit Residuals\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()

fig.show()

print(fits)

In [ ]:
fig,ax = plt.subplots(1,1)
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    
    ax.errorbar(x_data_dBm[1:], 2*(dAC[1:]-(func2(x_data_dBm[1:],*fits))), yerr = dAC_er[1:], fmt = ".", label=f"band {band}")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)

x_ac = 20*np.log10(varied[1]/scale)
for k in basis_t:
    ax.plot(x_ac, 1e-6*energies[k]/tau-2*func2(x_ac,*fits))
#ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
ax.legend()
ax.set_xlim(-20,max(x_ac))
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC}$ at 1190 (MHz)")
ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot 4780MHz$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()

In [ ]:
xdat = x_data_Vs[1:]
ydat = dAC_split[1:]
yer = dAC_CoM_er[1:]

func = lambda x, alpha, c: alpha*(np.sqrt(c**2+x**2)-c)
guess= [4.76e2, 2e-1]

popt,pcov = curve_fit(func,xdat,ydat,sigma=yer,p0=guess)
perr = np.sqrt(np.diag(pcov))

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat,yerr=yer,fmt=".")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
#ax.plot(xlin,func(xlin,*popt))
ax.plot(xlin,func(xlin,*guess))
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func(xdat,*guess),yerr=yer,fmt=".")
fig.show()

In [ ]:
def res(xdat,ydat,func,fit):
    return np.mean(ydat-func(xdat,*fit))

def res_rms(xdat,ydat,func,fit):
    return np.mean(np.sqrt(((ydat-func(xdat,*fit))**2).sum()))

alphas = np.linspace(2.2,3.1,100)*1e2/2
cs = np.linspace(0.05,0.12,100)
residuals = np.zeros((len(alphas),len(cs)),dtype=float)
residuals_rms = np.zeros((len(alphas),len(cs)),dtype=float)
for i, alpha in enumerate(alphas):
    for j, c in enumerate(cs):
        residuals[i,j] = abs(res(xdat,ydat,func,[alpha,c]))
        residuals_rms[i,j] = res_rms(xdat,ydat,func,[alpha,c])

In [ ]:
fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,

]
im = ax.imshow(residuals, interpolation="none", aspect="auto", extent=extent)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
fig.show()

fig,ax = plt.subplots(1,1)
alphas_step = (max(alphas)-min(alphas))/len(alphas)
cs_step = (max(cs)-min(cs))/len(cs)
extent = [
    min(cs)-cs_step/2,
    max(cs)+cs_step/2,
    max(alphas)+alphas_step/2,
    min(alphas)-alphas_step/2,

]
im = ax.imshow(residuals_rms, interpolation="none", aspect="auto", extent=extent, vmin=np.min(residuals_rms), vmax=np.min(residuals_rms)+10)
fig.colorbar(im,ax=ax, use_gridspec=True, shrink=.7)
ax.set_xlabel("c")
ax.set_ylabel("$\\alpha'$")
ax.set_xticks(cs.astype(float),minor=2)
ax.set_yticks(alphas.astype(float),minor=2)
fig.tight_layout()
fig.show()

In [ ]:
np.argwhere(residuals_rms==np.min(residuals_rms))[0]

In [ ]:
fits = [alphas[49],cs[46]]

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat,yerr=yer,fmt=".")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func(xlin,*fits))
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func(xdat,*fits),yerr=yer,fmt=".")
fig.show()

In [ ]:
print(fits)

In [ ]:
func2 = lambda x, alpha, c: alpha*(np.sqrt(c**2+10**(x/10)) - c)

fits = [-854.17,0.1138]
xdat = x_data_dBm[1:]
ydat = dAC_CoM[1:]
yer = dAC_CoM_er[1:]

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat,yerr=yer,fmt=".")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)
ax.plot(xlin,func2(xlin,*fits))
fig.show()

fig,ax = plt.subplots(1,1)
ax.errorbar(xdat,ydat-func2(xdat,*fits),yerr=yer,fmt=".")
fig.show()

In [ ]:
func2(-30,*fits)+40.5

In [ ]:
bands = range(1,6)
x_data_dBm = res_df["RF power (dBm)"]
x_data_Vs = np.sqrt(10**(x_data_dBm/10))
y_str = "mu {} (MHz)"
er_str = "sig {} (MHz)"
bare_res = res_df["mu 1 (MHz)"][0]
fig, ax = plt.subplots(1,1, figsize=(8,8))
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = 2*(measured_res-bare_res)
    dAC_er = 2*errb
    ax.errorbar(x_data_Vs, dAC, yerr = dAC_er, fmt = ".")
ax.set_title("AC stark shifts on $|52P_{3/2}$ induced by 4780MHz Dressing field")
ax.set_xlabel("Dressing Field Strength (arb)")
ax.set_ylabel("AC stark shift on $|52P_{3/2}>$ band (MHz)")
fig.tight_layout()
fig.show()
fig.savefig("Unscaled Wiki AC stark shifts.png")

In [ ]:
energies[basis_t].mean(0)

In [ ]:
levels, basis, basis_n = None, None, None

In [ ]:
fields = {
    "Eac": np.append(np.linspace(0,22,200), np.linspace(22+22/200,100,200)),
#    "Eac": 80,
#    "ellipticity": np.linspace(0,np.sqrt(0.04),200)**2,
    "ellipticity": 0.023,
    "field_omega": tau*4780e6,
    "Edc": 0.0,
    "theta": 0
}

for key, value in fields.items():
    try:
        tmp = value[1]
    except TypeError:
        continue
    else:
        varied = (key, value)

t_level = RydStateFS(52,1,3/2)
max_det = tau*30e9
dl = 1
n_max = 3

comp = {"n_max": n_max, "dl": dl, "max_det": max_det}

if levels is None:
    levels, basis = build_basis(t_level, max_det, single_side=False, dl=dl)
    basis_n = build_n_basis(basis, n_max)

In [ ]:
H0 = build_H0(basis, t_level, offset_zeeman=1e6)
energies, eigenstates = floquet_loop(basis, H0, n_max=n_max, varied=varied, **fields)

In [ ]:
thrs = 0.5
fig, ax = plt.subplots(1,1, figsize=(5,5))

basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
interesting_inds = []
for t_ind in basis_t:
    for k, ps in enumerate(eigenstates[:, t_ind, :]):
        if max(np.absolute(ps)**2) > thrs:
            interesting_inds.append(k)

interesting_inds = list(set(interesting_inds))
scale = 79.5/0.474
for k in interesting_inds:
    ax.plot(x_ac, 1e-6*np.real(energies[k,:]-energies[k,0]*0)/tau,label = basis[k%len(basis)].ket())

scale = 79.5/0.474
for band in range(n_bands):
    ax.errorbar(x_data, dACs, yerr = dACs_er, fmt = ".", label=f"band {band}")

ax.set_title(f"AC stark shifts\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$; $\epsilon = {fields['ellipticity']:.3f}$; $P_{{scale}}={scale:.0f}V/m$")
ax.set_ylabel("$\Delta_{AC}(MHz)$")
ax.set_xlabel("Dressing Power (dBm)")
#ax.set_ylim(-900,-80)
ax.set_xlim(-20,max(x_ac))
#ax.set_xscale("log")
#ax.set_yscale("symlog")
#ax.set_title(f"Bands wrt Ellipticity\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$\nEac = {fields['Eac']:.3f} V/m\n")
#ax.set_ylabel(f"Band Energy - $E_{{{t_level.ket()}}}$ (MHz)")
#ax.set_xlabel("Dressing Field Ellipticity")
ax.legend()
for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()
#fig.savefig(f"Theory-Experiment_handscale_ellipticity_{fields['ellipticity']:.3f}.png")#
#fig.savefig(f"Band Frequency ellipticity.png")

In [ ]:
fig,ax = plt.subplots(1,1)
for band in bands:
    measured_res = res_df[y_str.format(band)]
    errb = res_df[er_str.format(band)]
    dAC = (measured_res-bare_res)
    dAC_er = errb
    if band == 1:
        ax.errorbar(x_data_dBm[1:], 2*(dAC[1:]-(func2(x_data_dBm[1:],*fits))), yerr = dAC_er[1:], fmt = ".", label=f"measured resonances")
    else:
        ax.errorbar(x_data_dBm[1:], 2*(dAC[1:]-(func2(x_data_dBm[1:],*fits))), yerr = dAC_er[1:], fmt = ".")
#ax.errorbar(xdat,ydat+dAC_split[1:]/2,fmt=".")
#ax.errorbar(xdat,ydat-dAC_split[1:]/2,fmt=".")
xlin = np.linspace(min(xdat),max(xdat),1000)

scale = 79.5/0.473
x_ac = 20*np.log10(varied[1]/scale)
for k in basis_t:
    if k == min(basis_t):
        ax.plot(x_ac, 1e-6*energies[k]/tau-2*func2(x_ac,*fits), label = "computed resonances")
    else:
        ax.plot(x_ac, 1e-6*energies[k]/tau-2*func2(x_ac,*fits))
#ax.plot(xlin,func2(xlin,*fits), label="Fit to CoM")
ax.legend()
ax.set_xlim(-20,max(x_ac))
ax.set_xlabel("Dressing Power (dBm)")
ax.set_ylabel("$\Delta_{AC,i}$ - Mean (MHz)")
field_omega=fields["field_omega"]
ellipticity = fields["ellipticity"]
ax.set_title(f"AC stark shift CoM fit\n$\omega_{{d}}=2\pi \cdot {field_omega*1e-6/tau:.0f}MHz$; $\epsilon = {ellipticity}$; $P_{{scale}} = {scale:.0f}V/m$")

for txt in [ax.xaxis.label, ax.yaxis.label, ax.title]:
    txt.set_fontsize("14")
fig.tight_layout()
fig.show()

In [ ]:
k
len()

In [ ]:
eps = np.arange(0,0.045,0.005)
for ellip in eps:
    ellip = float(ellip)
    fields = {
        "Eac": np.append(np.linspace(0,22,200), np.linspace(22+22/200,100,200)),
        "ellipticity": ellip,
        "field_omega": tau*4780e6,
        "Edc": 0.0,
        "theta": 0
    }

    for key, value in fields.items():
        try:
            tmp = value[1]
        except TypeError:
            continue
        else:
            varied = (key, value)

    H0 = build_H0(basis, t_level, offset_zeeman=1e6)
    energies, eigenstates = floquet_loop(basis, H0, n_max=n_max, varied=varied, **fields)

    thrs = 0.5
    fig, ax = plt.subplots(1,1, figsize=(8,8))

    basis_t = [i+len(basis)*n_max for i, state in enumerate(basis) if [state.n, state.l, state.j] == [t_level.n, t_level.l, t_level.j]]
    interesting_inds = []
    for t_ind in basis_t:
        for k, ps in enumerate(eigenstates[:, t_ind, :]):
            if max(np.absolute(ps)**2) > thrs:
                interesting_inds.append(k)

    interesting_inds = list(set(interesting_inds))
    for k in interesting_inds:
        ax.plot(varied[1], 1e-6*np.real(energies[k,:]-energies[k,0])/tau, label = basis_n[k].ket())

    scale = 79/0.474
    for band in bands:
        measured_res = res_df[y_str.format(band)]
        errb = res_df[er_str.format(band)]
        dAC = 2*(measured_res-bare_res)
        dAC_er = 2*errb
        x_data_scaled = x_data_Vs*scale
        ax.errorbar(x_data_scaled, dAC, yerr = dAC_er, fmt = ".", label=f"band {band}")

    ax.set_title(f"AC stark shifts\n$\omega_D = {fields['field_omega']*1e-6/tau:.0f} MHz$\n$\epsilon = {fields['ellipticity']:.3f}$")
    ax.set_ylabel("AC stark shift (MHz)")
    ax.set_xlabel("AC field strength (V/m)")
    ax.legend()
    fig.tight_layout()
    fig.show()
    fig.savefig(f"Theory-Experiment_handscale_ellipticity_{fields['ellipticity']:.3f}.png")

In [ ]:
key

In [ ]:
type(value)

In [ ]:
basis_t

In [ ]:
basis[basis_t[3]].ket()